**This file contains an Ensembling(Blending Approach) which is done by combining 6 different tree based models**

In [ ]:
from google.colab import files
files=files.upload()

Saving train_df_reg_users.csv to train_df_reg_users.csv


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
pip install optuna

In [ ]:
pip install catboost

**Hyperparameter Tuning is done using optuna**

**Six different tree based models are used as base model followed by a linear regression model which efficiently finds the weights for the combination of different models.these models when combined produces an effectively better score which puts us in top 1% at kaggle leaderboard.**

1)RandomForest Regressor

2)XGBRegressor

3)LGBMRegressor

4)GRADIENTBOOSTINGRegressor

5)CATBOOSTRegressor

6)ADABOOSTRegressor




In [ ]:
import optuna
#from lofo import LOFOImportance,Dataset,plot_importance
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression,Lasso,ElasticNet
from sklearn import tree
import xgboost
import lightgbm
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [ ]:
train_df=pd.read_csv('train_df_reg_users.csv')

In [ ]:
validation_set=train_df[(train_df['day']>=18) & (train_df['day']<=19)] 

In [ ]:
train_df.drop(index=list(validation_set.index),axis=0,inplace=True)

**Dropping off the columns which are not required for our predictions.**

In [ ]:
y_train=np.log1p(train_df['registered'])
train_df.drop(['datetime','casual','count','registered','month','day','increasing_reg_users_2011'],axis=1,inplace=True)

In [ ]:
y_valid=np.log1p(validation_set['registered'])
validation_set.drop(['datetime','casual','count','registered','month','day','increasing_reg_users_2011'],axis=1,inplace=True)

In [ ]:
def objective(trial):

  x,y=train_df,y_train
  #n_estimators=int(trial.suggest_discrete_uniform('n_estimators',100,1200,100))
  max_depth=int(trial.suggest_discrete_uniform('max_depth',10,30,1))
  #subsample=trial.suggest_uniform('subsample',0.7,0.9)
  #colsample_bytree=trial.suggest_uniform('colsample_bytree',0.5,0.6)
  #colsample_bylevel=trial.suggest_uniform('colsample_bylevel',0.7,0.9)

  clf=RandomForestRegressor(max_depth=max_depth)
  score=cross_val_score(clf,x,y,scoring='neg_root_mean_squared_error',n_jobs=-1,cv=3)
  rmsle_score=score.mean()

  return rmsle_score

In [ ]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=20)

[I 2021-09-07 06:10:15,177] A new study created in memory with name: no-name-fe0c2aef-6890-4cf7-9531-7f1f303cf444
[I 2021-09-07 06:10:20,853] Trial 0 finished with value: -0.5495739592440417 and parameters: {'max_depth': 17.0}. Best is trial 0 with value: -0.5495739592440417.
[I 2021-09-07 06:10:24,524] Trial 1 finished with value: -0.5682227197223652 and parameters: {'max_depth': 10.0}. Best is trial 0 with value: -0.5495739592440417.
[I 2021-09-07 06:10:30,676] Trial 2 finished with value: -0.5487336369149364 and parameters: {'max_depth': 30.0}. Best is trial 2 with value: -0.5487336369149364.
[I 2021-09-07 06:10:36,813] Trial 3 finished with value: -0.5465519502270861 and parameters: {'max_depth': 28.0}. Best is trial 3 with value: -0.5465519502270861.
[I 2021-09-07 06:10:40,503] Trial 4 finished with value: -0.569520200072491 and parameters: {'max_depth': 10.0}. Best is trial 3 with value: -0.5465519502270861.
[I 2021-09-07 06:10:45,728] Trial 5 finished with value: -0.550354331051

In [ ]:
trial=study.best_trial

In [ ]:
trial.params

{'max_depth': 30.0}

In [ ]:
trial.values

[-0.5440604384971298]

In [ ]:
rf_reg_rs=RandomForestRegressor(max_depth=30,n_estimators=100)
rf_reg_rs.fit(train_df,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=30, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
validation_pred=rf_reg_rs.predict(validation_set)
valid_rf_df=pd.DataFrame(validation_pred,columns=['valid_rf_pred'])

In [ ]:
#XGBREGRESSOR

In [ ]:
def objective(trial):

  x,y=train_df,y_train
  n_estimators=int(trial.suggest_discrete_uniform('n_estimators',1000,1200,20))
  #max_depth=int(trial.suggest_discrete_uniform('max_depth',3,5,1))
  subsample=trial.suggest_uniform('subsample',0.7,0.9)
  colsample_bytree=trial.suggest_uniform('colsample_bytree',0.5,0.6)
  colsample_bylevel=trial.suggest_uniform('colsample_bylevel',0.7,0.9)

  clf=XGBRegressor(n_estimators=n_estimators,subsample=subsample,colsample_bytree=colsample_bytree,colsample_bylevel=colsample_bylevel)
  score=cross_val_score(clf,x,y,scoring='neg_root_mean_squared_error',n_jobs=-1,cv=3)
  rmsle_score=score.mean()

  return rmsle_score

In [ ]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=20)

[I 2021-09-07 05:40:40,636] A new study created in memory with name: no-name-ada57ca9-0a66-4038-ab93-7e5b91a420c3
[I 2021-09-07 05:40:49,449] Trial 0 finished with value: -0.38358271867567223 and parameters: {'n_estimators': 1200.0, 'subsample': 0.7333151587974961, 'colsample_bytree': 0.5496011189374541, 'colsample_bylevel': 0.8339733788204824}. Best is trial 0 with value: -0.38358271867567223.
[I 2021-09-07 05:40:56,692] Trial 1 finished with value: -0.39144053147036795 and parameters: {'n_estimators': 1060.0, 'subsample': 0.8600344201725152, 'colsample_bytree': 0.5492301686882016, 'colsample_bylevel': 0.786113200325492}. Best is trial 0 with value: -0.38358271867567223.
[I 2021-09-07 05:41:04,700] Trial 2 finished with value: -0.3948145465607458 and parameters: {'n_estimators': 1180.0, 'subsample': 0.8850788285356972, 'colsample_bytree': 0.5745018260404849, 'colsample_bylevel': 0.7614845871232951}. Best is trial 0 with value: -0.38358271867567223.
[I 2021-09-07 05:41:12,064] Trial 3 

In [ ]:
trial=study.best_trial

In [ ]:
trial.value

-0.3828540846279318

In [ ]:
trial.params

{'colsample_bylevel': 0.7204774929730215,
 'colsample_bytree': 0.5596404657324598,
 'n_estimators': 1020.0,
 'subsample': 0.7326449298976951}

In [ ]:
xgb_reg_modified=XGBRegressor(n_estimators=1020,subsample=0.732,colsample_bytree=0.559,colsample_bylevel=0.72)
xgb_reg_modified.fit(train_df,y_train)

[06:17:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.72,
             colsample_bynode=1, colsample_bytree=0.559, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1020,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.732, verbosity=1)

In [ ]:
validation_pred=xgb_reg_modified.predict(validation_set)
valid_xgb_df=pd.DataFrame(validation_pred,columns=['valid_xgb_pred'])

In [ ]:
#LIGHTGBM

In [ ]:
def objective(trial):

  x,y=train_df,y_train
 # n_estimators=int(trial.suggest_discrete_uniform('n_estimators',1000,1200,20))
  max_depth=int(trial.suggest_discrete_uniform('max_depth',5,11,1))
  #subsample=trial.suggest_uniform('subsample',0.7,0.9)
  colsample_bytree=trial.suggest_uniform('colsample_bytree',0.4,0.5)
  #colsample_bylevel=trial.suggest_uniform('colsample_bylevel',0.7,0.9)

  clf=LGBMRegressor(max_depth=max_depth,colsample_bytree=colsample_bytree)
  score=cross_val_score(clf,x,y,scoring='neg_root_mean_squared_error',n_jobs=-1,cv=3)
  rmsle_score=score.mean()

  return rmsle_score

In [ ]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=30)

[I 2021-09-07 05:44:14,602] A new study created in memory with name: no-name-f4bd3ac1-b8ec-4b1f-b145-38cc68a204eb
[I 2021-09-07 05:44:15,122] Trial 0 finished with value: -0.41420750132702455 and parameters: {'max_depth': 11.0, 'colsample_bytree': 0.478168315214494}. Best is trial 0 with value: -0.41420750132702455.
[I 2021-09-07 05:44:15,460] Trial 1 finished with value: -0.40679611271757316 and parameters: {'max_depth': 7.0, 'colsample_bytree': 0.4023737868188303}. Best is trial 1 with value: -0.40679611271757316.
[I 2021-09-07 05:44:15,785] Trial 2 finished with value: -0.4113610418393649 and parameters: {'max_depth': 11.0, 'colsample_bytree': 0.44327095448149406}. Best is trial 1 with value: -0.40679611271757316.
[I 2021-09-07 05:44:16,116] Trial 3 finished with value: -0.4113610418393649 and parameters: {'max_depth': 11.0, 'colsample_bytree': 0.41228680802783674}. Best is trial 1 with value: -0.40679611271757316.
[I 2021-09-07 05:44:16,410] Trial 4 finished with value: -0.40486107

In [ ]:
trial=study.best_trial

In [ ]:
trial.value

-0.40486107796320553

In [ ]:
trial.params

{'colsample_bytree': 0.4697913350723262, 'max_depth': 5.0}

In [ ]:
lgb_reg_modified=LGBMRegressor(n_estimators=100,max_depth=5,colsample_bytree=0.469)
lgb_reg_modified.fit(train_df,y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.469,
              importance_type='split', learning_rate=0.1, max_depth=5,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
validation_pred=lgb_reg_modified.predict(validation_set)
valid_lgb_df=pd.DataFrame(validation_pred,columns=['valid_lgb_pred'])

In [ ]:
#GRADIENTBOOSTINGREGRESSOR

In [ ]:
def objective(trial):

  x,y=train_df,y_train
  n_estimators=int(trial.suggest_discrete_uniform('n_estimators',300,400,10))
  #max_depth=int(trial.suggest_discrete_uniform('max_depth',3,15,1))
  subsample=trial.suggest_uniform('subsample',0.5,0.55)
  #colsample_bytree=trial.suggest_uniform('colsample_bytree',0.5,0.6)
  #colsample_bylevel=trial.suggest_uniform('colsample_bylevel',0.7,0.9)

  clf=GradientBoostingRegressor(n_estimators=n_estimators,subsample=subsample)
  score=cross_val_score(clf,x,y,scoring='neg_root_mean_squared_error',n_jobs=-1,cv=3)
  rmsle_score=score.mean()

  return rmsle_score

In [ ]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=20)

[I 2021-09-07 05:52:13,115] A new study created in memory with name: no-name-e7ad595d-b4b7-4076-b665-ac7dded60a26
[I 2021-09-07 05:52:18,046] Trial 0 finished with value: -0.433199417587072 and parameters: {'n_estimators': 320.0, 'subsample': 0.5167132736687742}. Best is trial 0 with value: -0.433199417587072.
[I 2021-09-07 05:52:21,651] Trial 1 finished with value: -0.43725197209179106 and parameters: {'n_estimators': 320.0, 'subsample': 0.5326657066072495}. Best is trial 0 with value: -0.433199417587072.
[I 2021-09-07 05:52:24,937] Trial 2 finished with value: -0.4276888540711686 and parameters: {'n_estimators': 300.0, 'subsample': 0.5140424132703658}. Best is trial 2 with value: -0.4276888540711686.
[I 2021-09-07 05:52:29,391] Trial 3 finished with value: -0.4271413129943067 and parameters: {'n_estimators': 390.0, 'subsample': 0.5439249757709346}. Best is trial 3 with value: -0.4271413129943067.
[I 2021-09-07 05:52:33,289] Trial 4 finished with value: -0.410957637048974 and paramete

In [ ]:
trial=study.best_trial

In [ ]:
trial.value

-0.40864715463079443

In [ ]:
trial.params

{'n_estimators': 320.0, 'subsample': 0.5159483091415967}

In [ ]:
gb_reg_modified=GradientBoostingRegressor(n_estimators=360,max_depth=3,subsample=0.501)
gb_reg_modified.fit(train_df,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=360,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=0.501, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
validation_pred=gb_reg_modified.predict(validation_set)
valid_gb_df=pd.DataFrame(validation_pred,columns=['valid_gb_pred'])

In [ ]:
#CATBOOSTREGRESSOR

In [ ]:
def objective(trial):

  x,y=train_df,y_train
  n_estimators=int(trial.suggest_discrete_uniform('n_estimators',100,1200,100))
  max_depth=int(trial.suggest_discrete_uniform('max_depth',3,10,1))
  #subsample=trial.suggest_uniform('subsample',0.7,0.9)
  #colsample_bytree=trial.suggest_uniform('colsample_bytree',0.5,0.6)
  #colsample_bylevel=trial.suggest_uniform('colsample_bylevel',0.7,0.9)

  clf=CatBoostRegressor(max_depth=max_depth,n_estimators=n_estimators)
  score=cross_val_score(clf,x,y,scoring='neg_root_mean_squared_error',n_jobs=-1,cv=3)
  rmsle_score=score.mean()

  return rmsle_score

In [ ]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=15)

[I 2021-09-07 05:54:58,200] A new study created in memory with name: no-name-dc201c32-f9d4-483e-8c06-a362104a0332
[I 2021-09-07 05:55:14,483] Trial 0 finished with value: -0.532409063119348 and parameters: {'n_estimators': 500.0, 'max_depth': 10.0}. Best is trial 0 with value: -0.532409063119348.
[I 2021-09-07 05:55:16,679] Trial 1 finished with value: -0.45254660932458285 and parameters: {'n_estimators': 300.0, 'max_depth': 6.0}. Best is trial 1 with value: -0.45254660932458285.
[I 2021-09-07 05:55:50,340] Trial 2 finished with value: -0.5154996590266899 and parameters: {'n_estimators': 1100.0, 'max_depth': 10.0}. Best is trial 1 with value: -0.45254660932458285.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

[I 2021-09-07 05:55:51,140] Trial 3 finished with value: -0.41776699248598864 and parameters: {

In [ ]:
trial=study.best_trial

In [ ]:
trial.value

-0.41776699248598864

In [ ]:
trial.params

{'max_depth': 3.0, 'n_estimators': 100.0}

In [ ]:
cat_reg_modified=CatBoostRegressor(max_depth=3,n_estimators=100)
cat_reg_modified.fit(train_df,y_train)

Learning rate set to 0.336843
0:	learn: 1.1049205	total: 5.85ms	remaining: 579ms
1:	learn: 0.9205962	total: 8.53ms	remaining: 418ms
2:	learn: 0.8078039	total: 10.6ms	remaining: 342ms
3:	learn: 0.7213302	total: 13.4ms	remaining: 321ms
4:	learn: 0.6615261	total: 16.1ms	remaining: 306ms
5:	learn: 0.6232639	total: 18.9ms	remaining: 296ms
6:	learn: 0.5860473	total: 21.8ms	remaining: 289ms
7:	learn: 0.5651049	total: 24.7ms	remaining: 284ms
8:	learn: 0.5477575	total: 27.6ms	remaining: 279ms
9:	learn: 0.5152362	total: 30.5ms	remaining: 275ms
10:	learn: 0.4992060	total: 33.3ms	remaining: 270ms
11:	learn: 0.4886255	total: 36.2ms	remaining: 265ms
12:	learn: 0.4722805	total: 39ms	remaining: 261ms
13:	learn: 0.4588357	total: 41.9ms	remaining: 257ms
14:	learn: 0.4524359	total: 44.6ms	remaining: 253ms
15:	learn: 0.4485181	total: 47.4ms	remaining: 249ms
16:	learn: 0.4444079	total: 50.1ms	remaining: 245ms
17:	learn: 0.4404314	total: 53.7ms	remaining: 245ms
18:	learn: 0.4345321	total: 55.7ms	remaining: 

In [ ]:
validation_pred=cat_reg_modified.predict(validation_set)
valid_cat_df=pd.DataFrame(validation_pred,columns=['valid_cat_pred'])

In [ ]:
#ADABOOSTREGRESSOR

In [ ]:
def objective(trial):

  x,y=train_df,y_train
  #n_estimators=int(trial.suggest_discrete_uniform('n_estimators',100,1200,100))
  learning_rate=trial.suggest_uniform('learning_rate',0.001,1)

  clf=AdaBoostRegressor(learning_rate=learning_rate,base_estimator=XGBRegressor())
  score=cross_val_score(clf,x,y,scoring='neg_root_mean_squared_error',n_jobs=-1,cv=3)
  rmsle_score=score.mean()

  return rmsle_score

In [ ]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=10)

[I 2021-09-07 05:56:58,511] A new study created in memory with name: no-name-3e6fe830-ff0d-4fda-8874-bc4e30d57697
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

[I 2021-09-07 05:57:47,426] Trial 0 finished with value: -0.45999731670325456 and parameters: {'learning_rate': 0.01642671301594668}. Best is trial 0 with value: -0.45999731670325456.
[I 2021-09-07 05:58:35,040] Trial 1 finished with value: -0.44246496052231926 and parameters: {'learning_rate': 0.23176791792297807}. Best is trial 1 with value: -0.44246496052231926.
[I 2021-09-07 05:59:20,176] Trial 2 finished with value: -0.4416604641754485 and parameters: {'learning_rate': 0.3008989059848994}. Best is trial 2 with value: -0.4416604641754485.
[I 2021-09-07 05:59:46,604] Trial 3 finished with value: -0.440731866040166 and parameters: {'learning_ra

In [ ]:
trial=study.best_trial

In [ ]:
trial.params

{'learning_rate': 0.5231647012718974}

In [ ]:
trial.value

-0.440731866040166

In [ ]:
ada_reg_modified=AdaBoostRegressor(learning_rate=0.5231)
ada_reg_modified.fit(train_df,y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.5231, loss='linear',
                  n_estimators=50, random_state=None)

In [ ]:
validation_pred=ada_reg_modified.predict(validation_set)
valid_ada_df=pd.DataFrame(validation_pred,columns=['valid_ada_pred'])

In [ ]:
#Concatenating the validation predictions from all the models

In [ ]:
input_df_meta_model=pd.concat([valid_rf_df,valid_xgb_df,valid_lgb_df,valid_gb_df,valid_cat_df,valid_ada_df],axis=1)

In [ ]:
input_df_meta_model

,valid_rf_pred,valid_xgb_pred,valid_lgb_pred,valid_gb_pred,valid_cat_pred,valid_ada_pred
0,3.602919,3.107699,3.147317,3.339836,3.382409,3.853128
1,3.999025,3.892664,3.855758,3.832085,3.936522,3.972053
2,3.950889,3.903407,4.012655,3.927627,3.908479,4.015568
3,3.929613,3.984300,3.941107,3.943864,3.935871,3.972053
4,4.245578,4.292346,4.087881,4.175951,4.212225,3.972053
...,...,...,...,...,...,...
1132,5.938054,5.894590,5.924683,5.926718,5.942208,5.061882
1133,5.530420,5.637119,5.584972,5.608040,5.647738,5.036934
1134,5.184213,5.334210,5.248171,5.278625,5.439399,5.000079
1135,5.048523,5.105744,4.988011,5.059587,5.109228,4.975356


In [ ]:
meta_model=LinearRegression()
meta_model.fit(input_df_meta_model,y_valid)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
meta_model.coef_

array([ 0.19096114,  0.42948774,  0.16495856,  0.04283149,  0.22072533,
       -0.05188154])

In [ ]:
from google.colab import files
files=files.upload()

Saving test_df.csv to test_df.csv


In [ ]:
test_df=pd.read_csv('test_df.csv')

In [ ]:
test_df=test_df[['season', 'holiday', 'workingday', 'weather', 'temp', 'humidity','windspeed', 'hour', 'year', 'weekday','busy_hours_non_working_day', 'peak_hours_working_day', 'hour_lag_value', 'increasing_reg_users_2012']]

In [ ]:
test_rf_pred=pd.DataFrame(rf_reg_rs.predict(test_df),columns=['valid_rf_pred'])
test_xgb_pred=pd.DataFrame(xgb_reg_modified.predict(test_df,columns=['valid_xgb_pred'])
test_lgb_pred=pd.DataFrame(lgb_reg_modified.predict(test_df,columns=['valid_lgb_pred'])
test_gb_pred=pd.DataFrame(gb_reg_modified.predict(test_df,columns=['valid_gb_pred'])
test_cat_pred=pd.DataFrame(cat_reg_modified.predict(test_df),columns=['valid_cat_pred'])
test_ada_pred=pd.DataFrame(ada_reg_modified.predict(test_df),columns=['valid_ada_pred'])

In [ ]:
final_test_df=pd.concat([test_rf_pred,test_xgb_pred,test_lgb_pred,test_gb_pred,test_cat_pred,test_ada_pred],axis=1)

In [ ]:
final_test_df

,valid_rf_pred,valid_xgb_pred,valid_lgb_pred,valid_gb_pred,valid_cat_pred,valid_ada_pred
0,2.437948,2.602516,2.473207,2.428588,2.550540,2.108657
1,1.748737,1.707745,1.706671,1.669607,1.797013,1.737003
2,0.819001,1.149806,1.234665,1.188564,1.274343,1.609200
3,0.897829,0.858923,0.975576,0.884709,0.894288,1.609200
4,0.871802,0.854364,0.975576,0.936807,0.912251,1.609200
...,...,...,...,...,...,...
6488,4.949968,5.319016,5.142366,5.329100,5.292116,4.676080
6489,4.721731,4.952726,4.845341,4.973198,4.991570,4.676080
6490,4.536906,4.614211,4.632019,4.694315,4.724357,4.311596
6491,4.367715,4.272962,4.255192,4.308905,4.322804,4.311596


In [ ]:
final_test_pred=meta_model.predict(final_test_df)

In [ ]:
final_test_pred_reg_users=np.expm1(final_test_pred)

In [ ]:
final_test_df=pd.DataFrame({'blended_pred_reg_users':final_test_pred_reg_users})

In [ ]:
final_test_df.to_csv('final_blended_pred_reg_users_27.csv',index=False)

**Final prediction on the test data**

In [ ]:
final_test_df

,blended_pred_reg_users
0,11.558620
1,4.520707
2,1.956876
3,1.303925
4,1.302240
...,...
6488,182.694869
6489,130.867248
6490,99.877611
6491,70.359872


In [ ]:
my_file=open('meta_blended_model.txt','wb')
pickle.dump(meta_model,my_file)
my_file.close()

In [ ]:
final_test_df.to_csv('test_df_blended.csv',index=False)

In [ ]:
from google.colab import files
files=files.upload()

Saving test_df_blended.csv to test_df_blended.csv


In [ ]:
test_df_blended=pd.read_csv('test_df_blended.csv')